In [34]:
import os
import fnmatch
import subprocess
import re
import simpleMenu
import ConfigParser
import logging
from logging.handlers import RotatingFileHandler
import sys
import unicodedata
import csv

appShortName = 'fileStreamPortfolio'

In [20]:
class configuration(object):
    def __init__(self, configFile = os.path.expanduser('~/.config/'+appShortName+'/config.ini')):
        self.logger = logging.getLogger(__name__)
        self.configPath = os.path.dirname(configFile)
        self.configFile = configFile # os.path.join(configPath, 'config.ini')
        self.getConfig()
    
    def writeConfig(self):
        # write out all the defined preferences (self.prefs) to the config file
        self.logger.debug('writing configuration to file: {}'.format(self.configFile))
        for key in self.prefs:
            eval ("self.parser.set('{0}', '{1}', self.{1})".format(self.mainSection, key))
        try:
            self.parser.write(open(self.configFile, 'w'))
        except Exception as e:
            logging.error('Error writing configuration file: {}'.format(e))
    
    def printConfig(self):
        '''
        Prints configuration file for debugging'''
        for section in self.parser.sections():
            print 'Section: {0}'.format(section)
            for key in self.parser.options(section):
                print '{0} = {1}'.format(key, self.parser.get(section, key))
#         for key in self.prefs:
#             print '{0} ='.format(key), eval('self.{0}'.format(key))
    
    
    def getConfig(self, mainSection = 'Main'):
        '''
        Reads configuration file and sets the following attributes:
        '''
        
        self.parser = ConfigParser.SafeConfigParser()
        
        # required options in the 'Main' section
        self.mainSection = mainSection
        
        # required key: [method for getting, default value]
        self.prefs = {
                        'mountpoint': [self.parser.get, '/Volumes/GoogleDrive'],
                        'teamdrive': [self.parser.get, ''],
                        'gradefolders': [self.parser.get, './gradefolders.txt'],
                        'portfoliofolder': [self.parser.get, '']
        }

        # make sure a configuration path exists
        if len(self.parser.read(self.configFile)) <= 0:
            logging.warn('no configuration files found at: {}'.format(self.configFile))
            logging.debug('creating configuration files')
            try:
                os.makedirs(os.path.expanduser(self.configPath))
            except OSError as e:
                if e.errno != 17:
                    logging.critical(e)
                    sys.exit(1)
        
        # make sure there is a main section
        if not self.parser.has_section(self.mainSection):
            self.parser.add_section(self.mainSection)
        
        preferences = {}
        
        # read search for the expected preferences in the configuration file
        # note which are missing and set to the default values above
        for key in self.prefs:
            try:
                preferences[key] = self.prefs[key][0](self.mainSection, key)
            except (ConfigParser.NoSectionError, ConfigParser.NoOptionError):
                self.parser.set(self.mainSection, key, self.prefs[key][1])
                preferences[key] = self.prefs[key][1]
                
        # set the values from the config
        for key in self.prefs:
            exec ("self.{0} = preferences['{0}']".format(key))
        
#         print preferences
#         print 'teamdrive', self.teamdrive
#         print 'mountpoint', self.mountpoint
#         print 'portfoliofolder', self.portfoliofolder
#         print 'gradefolders', self.gradefolders

In [21]:
class teamDrives(object):
    '''
    make working with google Team Drives through filestream a little easier
    '''

    def __init__(self, mountpoint='/Volumes/GoogleDrive/Team Drives/'):
        self.logger = logging.getLogger(__name__)
        self.mountpoint = mountpoint
        self.getDrives()
    
    def getDrives(self):
        self.logger.debug('Searching for Google Drive File Stream Mount Points')
        self.drives = {}
        try:
            drives = next(os.walk(self.mountpoint))[1]
        except Exception as e:
            self.logger.critical('error retriving list of Team Drives: {0}'.format(e))
            self.logger.critical('is Google Drive File Stream application running and configured?')
            self.logger.critical('mount point: {} is not accessible'.format(self.mountpoint))
            self.logger.critical('exiting')
            raise os.error('Error in os.walk for mount point: {0}'.format(self.mountpoint))
        for drive in drives:
            self.drives[drive] = oct(os.stat(self.mountpoint+drive).st_mode & 0o777)
    
    def listrwDrives(self):
        rwDrives = []
        for drive in self.drives:
            if 777 - int(self.drives[drive]) < 277:
                rwDrives.append(drive)
        return(sorted(rwDrives))
    
    def listFolders(self, teamDrive):
        try:
            folders = next(os.walk(self.mountpoint+teamDrive))[1]
        except Exception as e:
            self.logger.critical('Error getting list of Team Drives: {0}'.format(e))
            self.logger.critical('Is Google Drive File Stream application running and configured?')
            raise os.error('Error in os.walk for mount point: {0}'.format(self.mountpoint))
        return(folders)
    
    def find(self, pattern, teamDrive):
        result = []
        for root, dirs, files in os.walk(self.mountpoint+teamDrive):
            for name in dirs:
                if fnmatch.fnmatch(name, pattern):
                    result.append(os.path.join(root, name))
        return result
                
                
    

        

In [22]:
def checkFSMount(mountpoint = '/Volumes/GoogleDrive'):
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    logger.debug('Searching for Google Drive File Stream Mount Points')
    # list the local partitions (including FUSE partitions)
    mount = subprocess.check_output(['df', '-hl'])
    mountLines = mount.split('\n')
    mountPoints = []
    
    # makek a list of all the partitions
    for line in mountLines:
        try:
            # re extract anything that looks like a mount point
            mountSearch = re.search('\s+(\/[\S+]{0,})$', line)
            mountPoints.append(mountSearch.group(1))
        # ignore anything that doesn't match the re
        except Exception:
            pass    
    # check for mount point and try to launch the google drive file stream app
    if mountpoint in mountPoints:
        logger.debug('Found what appears to be a valid mountpoint at: {0}'.format(mountpoint))
        return True
    else:
        logger.info('Google Drive File Stream appears to not be running')
        return False
    


In [23]:
def get_valid_filename(s):
    """
    Return the given string converted to a string that can be used for a clean
    filename. Remove leading and trailing spaces
    """
    s = s.strip()
    s = strip_accents(s)
    return re.sub(r'(?u)[^-\w., ]', '', s)

In [24]:
def strip_accents(s):
    s = unicode(s, "utf-8")
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


In [ ]:
def fileRead(fname):
    '''
    read a file into a list, strip out all accented and special characters, leading spaces
    '''
    lines = []
    try:
        with open(fname) as f:
            for each in f:
                each = get_valid_filename(each)
                lines.append(each.strip('\n'))
            return(lines)
    except (OSError, IOError) as e:
        print 'error reading file:', fname, e
        return(False)

In [76]:
def parseCSVx(fname, expected = ['ClassOf', 'LastFirst', 'Student_Number']):
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    logger.info('parsing CSV File: {}'.format(fname))

    # raw CSV
    studentCSV = []
    # headers that were not inlcuded
    missingHeaders = []
    # map of positions in CSV
    headerMap = {}
        
    # read the csv file in Universal newline mode (rU)    
    try:
        with open(fname, 'rU') as csvfile:
            csvreader = csv.reader(csvfile)
            for row in csvreader:
                studentCSV.append(row)
    except (OSError, IOError) as e:
        logger.critical('error reading file {}\n{}'.format(fname, e))
        return(false)
    
    if len(studentCSV) > 1:
        logging.info('{} student records found'.format(len(studentCSV)))
    else:
        logging.critical('no student records found.')
        return(False)
    
    # check for the expected headers
    # pop the headers from the list
    
    csvHeader = studentCSV.pop(0)
    
    for each in expected:
        if not each in csvHeader:
            missingHeaders.append(each)
        
    if len(missingHeaders) > 0:
        logging.critical('error in student record file: {}'.format(fname))
        logging.critical('The following fileds were missing:')
        logging.critical('{:>5}'.format(missingHeaders))
        logging.critical('Please re-run the export and ensure that all of the following headers are included:')
        for each in expected:
            logging.critical('{:>5}'.format(each))
            return(False)

    # map headers to their index
    for index, value in enumerate(csvHeader):
        headerMap[value]=index
    
    for student in studentCSV:
        print student[headerMap['LastFirst']]
    
    

In [161]:
class parseCSV(object):

    def __init__(self, filename = False, headers = []):
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.DEBUG)

        self.filename = filename
        self.expectedHeaders = headers
        self.headerMap = {}
        self.csvList = False
        self.readCSV(self.filename)
        
    def readCSV(self, filename):
        # allow an parser object to be created with no filename
        if not filename:
            return() 
        self.logger.info('parsing CSV {0}'.format(filename))
        # raw CSV
        csvList = []

        # map of positions in CSV
        headerMap = {}

        # read the csv file in Universal newline mode (rU)    
        try:
            with open(filename, 'rU') as csvfile:
                csvreader = csv.reader(csvfile)
                for row in csvreader:
                    csvList.append(row)
        except (OSError, IOError) as e:
            self.logger.critical('error reading file {}\n{}'.format(self.filename, e))
            return(False)

        # if len > 1, set the object properties
        if len(csvList) > 1:
            self.logger.info('{} student records found'.format(len(csvList)-1))
            self.filename = filename
            self.csvList = csvList
        else:
            self.logger.critical('no student records found.')
            return(False)
        
        self.mapHeaders()

        
    def mapHeaders(self):
        # headers that were not inlcuded
        missingHeaders = []
        
        # pop the headers from the list
        csvHeader = self.csvList.pop(0)
        
        
        # check for the expected headers
        for each in self.expectedHeaders:
            if not each in csvHeader:
                missingHeaders.append(each)

        if len(missingHeaders) > 0:
            logger.critical('error in student record file: {}'.format(fname))
            logger.critical('The following fileds were missing:')
            logger.critical('{:>5}'.format(missingHeaders))
            logger.critical('Please re-run the export and ensure that all of the following headers are included:')
            for each in expected:
                logging.critical('{:>5}'.format(each))
                return(False)

        # map headers to their index
        for index, value in enumerate(csvHeader):
            self.headerMap[value]=index
            
    def lookup(self, index = 0, key = 0):
        return(self.csvList[index][self.headerMap[key]])
    


In [151]:
foo = parseCSV(filename = './student_export.text', headers = ['ClassOf', 'LastFirst', 'Student_Number'])
foo.csvList[0][foo.headerMap['ClassOf']]
for index, value in enumerate(foo.csvList):
    print index
    print (foo.lookup(index, 'ClassOf'))
    

INFO:__main__:parsing CSV ./student_export.text
INFO:__main__:3 student records found


0
3099
1
9990


In [164]:
def main():

    #### TESTING VARIABLES #####
    # remove these! and set cfg file with argv or something similar
    # config file
    cfgfile = './'+appShortName+'/config.ini'
    
    studentFile = './student_export.text'
    
    
    
    # //end remove
    #### TESTING VARIABLES #####
    
    # Create the Logger
    logger = logging.getLogger(__name__)

    for each in range (0, len(logger.handlers)):
        logger.removeHandler(logger.handlers[0])

    datefmt = '%y-%m-%d %H:%M:%S'

    logger.setLevel(logging.INFO)

    # file handler
#     fileformat = '%(asctime)s %(levelname)s %(module)s - %(funcName)s: %(message)s'
#     file_handler = logging.FileHandler(appShortName+'.log')
#     file_handler.setLevel(logging.DEBUG)
#     file_handler.setFormatter(logging.Formatter(fmt = fileformat, datefmt = datefmt))

    # stream handler
    streamformat = '%(asctime)s %(levelname)s - %(funcName)s: %(message)s'
    stream_handler = logging.StreamHandler(sys.stderr)
    stream_handler.setLevel(logging.CRITICAL)
    stream_handler.setFormatter(logging.Formatter(fmt = streamformat, datefmt = ''))

    # rotation handler
    fileformat = '%(asctime)s %(levelname)s - %(funcName)s: %(message)s'
    rotation_handler = RotatingFileHandler(appShortName+'.log', maxBytes = 50000, backupCount = 5)
    rotation_handler.setLevel(logging.DEBUG)
    rotation_handler.setFormatter(logging.Formatter(fmt = fileformat, datefmt = datefmt))


    # add handler to logger
    #logger.addHandler(file_handler)
    logger.addHandler(stream_handler)
    logger.addHandler(rotation_handler)
    logger.info('===================== Starting Log =====================')
        
    if checkFSMount():
        pass
    else:
        logger.info('Attempting to start Google Drive File Stream')
        try:
            gDriveFS = subprocess.check_call(["open", "-a", "Google Drive File Stream"])
        except subprocess.CalledProcessError as err:
            logger.warn('OS Error: {0}'.format(err))
            logger.critical('Google Drive File Stream does not appear to be installed. Please download from the link below')            
            logger.critical('https://support.google.com/drive/answer/7329379?hl=en')
            logger.critical('exiting')
            return(0)
        if checkFSMount():
            pass
        else:
            print "exiting"
            return(0)

    # get the configuration file
    myConfig = configuration(cfgfile)
    logger.info('=== Current Configuration Settings ===')
    for key in myConfig.prefs:
        try:
            logger.info('{0} -- {1}'.format(key, eval('myConfig.{0}'.format(key))))
        except Exception as e:
            pass
    
    # set the file stream drives object
    myDrives = teamDrives()
    
    # General purpose retry menu
    retryMenu = simpleMenu.menu(name = 'retry', items = ['Yes', 'No', 'Quit'])
    
    def getTeamDrive():
        '''ask for the appropriate team drive to search'''
        logger.debug('getting Team Drive name')
        try:
            myDrives.getDrives()
        except Exception as e:
            logger.critical(e)
            return(0)
        if len(myDrives.listrwDrives()) < 1:
            logger.critical('No Team Drives with write permissions available; exiting')
            return(0)
        else:
            rwDrivesMenu = simpleMenu.menu(name = 'Team Drives', items = myDrives.listrwDrives())        
            myDrive = rwDrivesMenu.loopChoice(optional = True, message = 'Which Team Drive contains the portfolio folder?')
 
            if myDrive is 'Q':
                print 'exiting'
                return(1)
            return(myDrive)
    
    def getPortfolioFolder():
        '''ask for appropriate portfolio folder'''
        logger.debug('beggining search for portfolio folder')
        print 'Searching in Team Drive: {0}'.format(myConfig.teamdrive)
        folderSearch = raw_input('Please enter part of the portfolio folder name (case sensitive search): ')
        logger.debug('searching TD \"{0}\" for \"{1}\"'.format(myConfig.teamdrive, folderSearch))
        fileList = myDrives.find(pattern = '*'+folderSearch+'*', teamDrive = myConfig.teamdrive)
        logger.debug('found {0} matches'.format(len(fileList)))
        if len(fileList) < 1:
            print 'No folders matching \"{0}\" found'.format(folderSearch)
            retry = retryMenu.loopChoice(optional = False, message = 'Would you like to try your search again?')
            if retry is 'Yes':
                getPortfolioFolder()
            if retry is 'No' or 'Quit':
                print 'exiting'
                return(1)
        
        # ask user to choose folder from list, quit or try again
        foldersMenu = simpleMenu.menu(name = 'Matching Folders', items = myDrives.find(
                                    pattern = '*'+folderSearch+'*', teamDrive = myConfig.teamdrive))
        myFolder = foldersMenu.loopChoice(optional = True, optchoices = {'Q': 'Quit', 'T': 'Try search with different folder name'},
                                        message = 'Which folder contains portfolios?')
        if myFolder is 'T':
            getPortfolioFolder()
        if myFolder is 'Q':
            print 'exiting'
            return(1)
                                                                         
        
        return(myFolder)
        
    def askContinue():
        logger.debug('prompting to continue')
        logger.debug('teamdrive: {0}'.format(myConfig.teamdrive))
        logger.debug('portfolio folder: {0}'.format(myConfig.portfoliofolder))
        continueMenu = simpleMenu.menu(name = 'Continue?', items = ['Yes', 'No: Set new team drive and folder'])
        response = continueMenu.loopChoice(optional = True, message = 'Continue with the portfolio folder: {0}/{1}'
                            .format(myConfig.teamdrive, myConfig.portfoliofolder))
        if response is 'Yes: Continue':
            logger.info('using portfolio folder: {0}{1}'.format(myConfig.teamdrive, myConfig.portfoliofolder))
            myConfig.writeConfig()
            return(0)
        if response is 'No: Set new team drive and folder':
            #need to do something about this; make some defs for get teamdrive and portofolio folder 
            myConfig.teamdrive = getTeamDrive()
            myConfig.portfoliofolder = getPortfolioFolder()
            askContinue()
        if response is 'Q':
            print 'exiting'
            return(1)
        
    if not myConfig.teamdrive:
        # if teamdrive is not in the configuration file, get it
        logger.info('no teamdrive set in configuration file')
        myConfig.teamdrive = getTeamDrive()
        if myConfig.teamdrive == 1:
            return(0)
    
    if not myConfig.portfoliofolder:
        # if portfolio folder is not in the configuration file, ask for it
        logger.info('no portfolio folder set in configuration file')
        myConfig.portfoliofolder = getPortfolioFolder()
        if myConfig.portfoliofolder == 1:
            return(0)
        
    if askContinue() == 1:
        return(0)
    
    logger.info('checking for alternative gradefolder.txt on Desktop')
    if os.path.exists(os.path.expanduser('~/Desktop/gradefolders.txt')):
        myConfig.gradefolders = os.path.expanduser('~/Desktop/gradefolders.txt')
        logger.info('set gradefolders path to: {}'.format(myConfig.gradefolders))
    else:
        logger.info('alternative not found; continuing with {0}'.format(myConfig.gradefolders))
        # check for grade folder description file
        
    if not os.path.exists(myConfig.gradefolders):
        logger.critical('{0} is missing \"gradefolders" option'.format(cfgfile))
        logger.critical('gradefolders should contain, one per line, folders to be created for each student: 00-Preeschool..12-Grade')
        logger.critical('please create a file named \"gradefolders.txt\" and place it on the Desktop')
        print 'exiting'
        return(1)
    
    # Open the gradefolders file and read all the lines into an array
    logger.info('opening grade folders file: {0}'.format(myConfig.gradefolders))
    # read in and sanitize the grade folders list (remove accents, invalid chars, etc)
    gradeFoldersList = fileRead(myConfig.gradefolders)
    if not gradeFoldersList:
        logger.critical('failed to open grade folders file')
        return(1)
    
    
#     if not os.path.exists(studentFile):
#         logger.critical('Student file: {0} does not exist or is unreadable'.format(studentFile))
#         return(1)
#     else:
#         logger.debug('reading studentFile: {}'.format(studentFile))
#         studentFileList = fileRead(studentFile)
#         if not studentFileList:
#             logger.critical('failed to open student file')
#             return(1)
#         if len(studentFileList) < 1:
#             logger.critical('Student file: {0} is empty'.studentFile)
#             return(1)
                
        
        
        
    # open the student data file and start creating folders as needed
    myCSV = parseCSV(studentFile)
    
    if not myCSV:
        logger.critical('error reading student file. exiting')
        
    # recurse myCSV.csvList, use lookup to pull ClassOf and LastFirst, sanitize them and then start creating folders
        
    

    
    
main()

INFO:__main__:===================== Starting Log =====================
DEBUG:__main__:Searching for Google Drive File Stream Mount Points
DEBUG:__main__:Found what appears to be a valid mountpoint at: /Volumes/GoogleDrive
INFO:__main__:=== Current Configuration Settings ===
INFO:__main__:teamdrive -- IT Blabla
INFO:__main__:mountpoint -- /Volumes/GoogleDrive
INFO:__main__:portfoliofolder -- /Volumes/GoogleDrive/Team Drives/IT Blabla/c/b/a/Portfolios
INFO:__main__:gradefolders -- ./gradefolders.txt
DEBUG:__main__:Searching for Google Drive File Stream Mount Points
DEBUG:__main__:prompting to continue
DEBUG:__main__:teamdrive: IT Blabla
DEBUG:__main__:portfolio folder: /Volumes/GoogleDrive/Team Drives/IT Blabla/c/b/a/Portfolios


Continue with the portfolio folder: IT Blabla//Volumes/GoogleDrive/Team Drives/IT Blabla/c/b/a/Portfolios
===== Continue? =====
 1) Yes
 2) No: Set new team drive and folder
 1 - 2 or {'Q': 'Quit'}: 1


INFO:__main__:checking for alternative gradefolder.txt on Desktop
INFO:__main__:alternative not found; continuing with ./gradefolders.txt
INFO:__main__:opening grade folders file: ./gradefolders.txt
INFO:__main__:parsing CSV ./student_export.text
INFO:__main__:2 student records found


In [ ]:
os.path.expanduser('~/Desktop/')